In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM
from  einops import rearrange


In [2]:
# load model and dataset
model = AutoModelForCausalLM.from_pretrained('qcw2333/YingLong_6m', trust_remote_code=True,torch_dtype=torch.bfloat16).cuda()
df = pd.read_csv("https://raw.githubusercontent.com/WenWeiTHU/TimeSeriesDatasets/refs/heads/main/ETT-small/ETTh2.csv")

# perpare input
lookback_length = 1440      # support the maximum context length up to 2880 
prediction_length = 96  
seqs = torch.tensor(df["OT"][:lookback_length]).unsqueeze(0).bfloat16().cuda()

# setup figure
fig, axs = plt.subplots(3, 1, sharex=True,figsize=(12, 6))


In [3]:

# vanilla forecsting
output = model.generate(seqs, future_token=prediction_length)
output = output.float().cpu().detach().numpy()

fig, axs = plt.subplots(3, 1, sharex=True,figsize=(12, 8))

axs[0].plot(df["OT"][:lookback_length + prediction_length], color="royalblue", label="Groundtruth")
mse =  np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length])**2)
mae =  np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length]).abs())
axs[0].plot(range(lookback_length, lookback_length + prediction_length), output[0,:,49], color="tomato", label=f"Vanilla")
axs[0].fill_between(range(lookback_length, lookback_length + prediction_length), output[0,:,9], output[0,:,90], color="tomato", alpha=0.30, label="80% prediction interval")
axs[0].legend()
axs[0].grid()
axs[0].set_title(f'MSE: {mse:.4f}, MAE: {mae:.4f}')


In [4]:

# DcoT forecasitng, i.e., predict longer future points
output = model.generate(seqs, future_token=4096)
output = output.float().cpu().detach().numpy()
output = output[:,:prediction_length,:]



axs[1].plot(df["OT"][:lookback_length + prediction_length], color="royalblue", label="Groundtruth")
mse =  np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length])**2)
mae =  np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length]).abs())
axs[1].plot(range(lookback_length, lookback_length + prediction_length), output[0,:,49], color="cyan", label=f"Dcot")
axs[1].fill_between(range(lookback_length, lookback_length + prediction_length), output[0,:,9], output[0,:,90], color="cyan", alpha=0.30, label="80% prediction interval")

axs[1].legend()
axs[1].grid()
axs[1].set_title(f'MSE: {mse:.4f}, MAE: {mae:.4f}')



In [3]:



# DcoT + Multi-Input Ensemble

res = 0
for lookback in [512,1024,1440]:
    
    x = torch.cat((seqs[:,-lookback:],-seqs[:,-lookback:]),dim = 0)
    res_tmp = model.generate(x, future_token=4096)
    res_tmp = rearrange(res_tmp, '(t b) l c -> b l c t', t = 2)
    res += res_tmp[...,0]  - res_tmp[...,1].flip(dims = [-1])
    
    
output = res / 6
output = output.float().cpu().detach().numpy()
output = output[:,:prediction_length,:]


axs[2].plot(df["OT"][:lookback_length + prediction_length], color="royalblue", label="Groundtruth")
err=   np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length])**2)
err1=  np.mean((output[0,:,49] - df["OT"][lookback_length:lookback_length + prediction_length]).abs())
axs[2].plot(range(lookback_length, lookback_length + prediction_length), output[0,:,49], color="gold", label=f"Dcot + multiple inputs")
axs[2].fill_between(range(lookback_length, lookback_length + prediction_length), output[0,:,9], output[0,:,90], color="gold", alpha=0.30, label="80% prediction interval")
axs[2].legend()
axs[2].grid()
axs[2].set_title(f'MSE: {err:.4f}, MAE: {err1:.4f}')




NameError: name 'plt' is not defined

In [4]:
plt.show()

NameError: name 'model' is not defined